In [ ]:
import os
import sys

path = os.getcwd()
while os.path.basename(os.path.normpath(path)) != 'roman-pandeia':
    path = os.path.dirname(path)
repo_path = path
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
from package.helpers import test_lens, test_physical_lens
from package.plots import plot
from hydra import initialize, compose
from glob import glob
import multiprocessing

from package.plots import diagnostic_plot, plot
from package.utils import util
from package.helpers import test_physical_lens, pyhalo, roman_params

plt.style.use(f'{repo_path}/package/mplstyle/science.mplstyle')

In [ ]:
with initialize(version_base=None, config_path='../../config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, repo_dir = config.machine.array_dir, config.machine.data_dir, config.machine.repo_dir

figure_dir = os.path.join(repo_dir, 'figures')
array_dir = os.path.join(array_dir, 'scenes_across_detectors')
util.create_directory_if_not_exists(array_dir)

# get Roman pixel scale
csv = os.path.join(repo_dir, 'data', 'roman_spacecraft_and_instrument_parameters.csv')
roman_pixel_scale = roman_params.RomanParameters(csv).get_pixel_scale()

In [ ]:
offset_list = [(0, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]

grid_oversample = 5
num_samples = 1000

buffer = 0.5  # arcseconds
side = 5.  # arcseconds
num_pix = round(side / roman_pixel_scale)
# if num_pix even, need to make it odd
if num_pix % 2 == 0:
    num_pix += 1

In [ ]:
# use test lens
lens = test_physical_lens.TestPhysicalLens()

# add CDM subhalos
# lens.add_subhalos(*pyhalo.generate_CDM_halos(lens.z_lens, lens.z_source))

model = lens.get_array(num_pix=num_pix * grid_oversample, side=side + buffer)

In [ ]:
# build Pandeia input
calc, _ = pandeia_input.build_pandeia_calc(csv=csv,
                                        array=model, 
                                        lens=lens, 
                                        band='f106',
                                        num_samples=num_samples)

# do Pandeia calculation        
image, execution_time = pandeia_input.get_pandeia_image(calc)

# save detector image
np.save(os.path.join(array_dir, f'test_physical_lens_image_{num_samples}'), image)

In [ ]:
cpu_count = multiprocessing.cpu_count()
generator = util.batch_list(lens_list, cpu_count)
batches = list(generator)

for batch in tqdm(batches):
    pool = Pool(processes=cpu_count) 
    output = []
    for each in pool.map(generate.main, batch):
        output.append(each)

In [ ]:
image_list, execution_times, num_point_sources = [], [], []

for tuple in output:
    (image, execution_time, num_ps) = tuple
    image_list.append(image)
    execution_times.append(execution_time)
    num_point_sources.append(num_ps)

In [ ]:
image_list = [i for i in image_list if i is not None]

plot.plot_grid(image_list, side=16)

In [ ]:
for image in image_list:
    np.save(os.path.join(array_dir, f'skypy_output_{str(i).zfill(5)}'), image)